# 카카오맵 API
## 도로명주소로부터 위도, 경도 정보 구하기
## [참고] https://developers.kakao.com/docs/latest/ko/local/dev-guide

In [2]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd

In [3]:
with open('kakaoapi.txt', mode='r') as key_file:
    kakao_key = key_file.read()

In [10]:
len(kakao_key)

32

In [7]:
addr = '서울특별시 성동구 금호동1가 산1-72'
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%84%B1%EB%8F%99%EA%B5%AC%20%EA%B8%88%ED%98%B8%EB%8F%991%EA%B0%80%20%EC%82%B01-72'

In [5]:
type(addr)

str

In [8]:
# "Authorization: KakaoAK {REST_API_KEY}"
result = requests.get(url, headers={"Authorization" : f"KakaoAK {kakao_key}"}).json()
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [18]:
lng = result['documents'][0]['x']
lat = result['documents'][0]['y']
lng, lat

('126.967109652664', '37.5598688323642')

## addr.csv로부터 주소를 읽어서 위도, 경도 정보를 추가한 뒤 addr2.csv에 저장

In [19]:
df = pd.read_csv('addr.csv')
df

,건물명,주소
0,한국경제신문사,서울특별시 중구 청파로 463 (중림동)
1,서울특별시청,서울특별시 중구 세종대로 110 (태평로1가)
2,종로구청,서울특별시 종로구 종로1길 36 (수송동)
3,중구청,서울특별시 중구 창경궁로 17 (예관동)
4,효창공원,서울특별시 용산구 효창원로 177-18 (효창동)


In [22]:
lng_list, lat_list = [], []
for i in df.index:
    url = f"{local_url}?query={quote(df['주소'][i])}"  # 주소의 i번째
    result = requests.get(url, headers={"Authorization" : f"KakaoAK {kakao_key}"}).json()
    lng = result['documents'][0]['x']
    lat = result['documents'][0]['y']
    lng_list.append(lng)
    lat_list.append(lat)

In [23]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,건물명,주소,위도,경도
0,한국경제신문사,서울특별시 중구 청파로 463 (중림동),37.5598688323642,126.967109652664
1,서울특별시청,서울특별시 중구 세종대로 110 (태평로1가),37.5663174209601,126.977829174031
2,종로구청,서울특별시 종로구 종로1길 36 (수송동),37.5735051436739,126.978988255925
3,중구청,서울특별시 중구 창경궁로 17 (예관동),37.5641201543296,126.998009728978
4,효창공원,서울특별시 용산구 효창원로 177-18 (효창동),37.544957405495,126.962290943049


In [24]:
df.to_csv('addr2.csv', index=False, sep=',',encoding='utf8')